# Radio Map

In [1]:
import pickle 
import json
import pandas as pd
import numpy as np
import collections

In [2]:
with open('RPs.pkl' , 'rb') as fp:
    d = pickle.load(fp)

In [3]:
with open('meta.json' , 'r') as fp:
    meta = json.load(fp)

In [4]:
'''
d has the following structure -> {location : RSSI value}


location - > room number.location in that room.timestamp.random
RSSI value -> a vector of length len(AP_LIST)
'''

'\nd has the following structure -> {location : RSSI value}\n\n\nlocation - > room number.location in that room.timestamp.random\nRSSI value -> a vector of length len(AP_LIST)\n'

In [5]:
AP_LIST = meta['ap_list']
AP_MAP = {AP_LIST[i] : i for i in range(len(AP_LIST))}
NUM_RECORDS = len(d)
DIRECTIONS =  ['north', 'south', 'east', 'west']

In [6]:
df = pd.DataFrame(AP_LIST, columns = ['AP'])
temp = pd.DataFrame(d.values()).transpose()
temp.columns = d.keys()
df = pd.concat([df, temp] , axis = 1)
df = df.set_index('AP')

In [7]:
df.describe()

,B7.1.north.1674507393.zjg0b,B7.1.north.1674507395.k26zy,B7.1.north.1674507397.pcipr,B7.1.north.1674507398.rs7qu,B7.1.north.1674507400.ouufp,B7.1.east.1674507403.ny1kt,B7.1.east.1674507405.76foq,B7.1.east.1674507407.l2cc6,B7.1.east.1674507408.ta03m,B7.1.east.1674507410.3yp8b,...,B5.4.south.1674507312.2xnka,B5.4.south.1674507314.v4bfk,B5.4.south.1674507315.y4pck,B5.4.south.1674507317.v9v01,B5.4.south.1674507319.9m61s,B5.4.west.1674507323.umd0n,B5.4.west.1674507324.9s3bx,B5.4.west.1674507326.zhzer,B5.4.west.1674507328.i9eco,B5.4.west.1674507329.ob8h2
count,38,59,66,74,85,86,91,95,97,98,...,81,83,86,89,89,89,87,92,96,100
unique,18,25,24,24,26,24,23,25,30,26,...,25,24,21,22,24,24,23,24,22,25
top,-83,-88,-88,-90,-88,-80,-89,-89,-89,-89,...,-87,-85,-92,-76,-90,-84,-90,-84,-89,-89
freq,5,7,5,8,8,8,8,10,10,10,...,7,7,9,9,8,10,11,9,10,10


In [8]:
# dd = pd.DataFrame(AP_LIST, columns = ['AP'])
# for k in list(d.keys()):
#     temp = pd.DataFrame(d[k], columns=[k])
#     dd = pd.concat([dd, temp] , axis = 1)
# dd = dd.set_index('AP')

In [9]:
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [10]:
RP_LIST = sorted(list(set([x[:find_nth(x, '.', 2)] for x in list(df.columns)])))
RP_MAP = {key : i for i, key in enumerate(RP_LIST)}
TIMESTEPS = 5

In [11]:
radio_map = {}

for direction in DIRECTIONS:
    arr = np.ndarray((len(AP_LIST), len(RP_MAP), TIMESTEPS), dtype = np.float16)
    for key in RP_MAP:
        cols = [x for x in df.columns if key + '.' + direction in x]
        temp = df[cols].to_numpy()
        arr[:,RP_MAP[key],:] = temp 

    radio_map[direction] = arr


In [12]:
radio_map['north'].shape

(953, 116, 5)

# Reliability and Similarity

In [13]:
GAMMA = -80  #Threshold above whcih an AP readig is too small
M = 2 #How many of the 5 time readings does the AP value need to be above GAMMA
LAMBDA = 1 / 1000 #Specifies a small amount to be added to hamming distance denominator to avoid zero division error

In [14]:
I = {} #Indicator matrices to check for "reliable" APs

In [15]:
for direction in DIRECTIONS:
    T = (radio_map[direction] > GAMMA).astype(int).sum(axis = 2)
    _I = (T > M).astype(int)
    I[direction] = _I

In [16]:
S = {} #Similarity matrices which stores a score for how "close" two RPs are

In [17]:
def hamming(a, b):
    return 1 / (np.count_nonzero(a!=b) + LAMBDA)

In [18]:
for direction in DIRECTIONS:
    _S = np.ndarray((len(RP_MAP) , len(RP_MAP)))
    _I = I[direction]
    for i in range(len(RP_MAP)):
        for j in range(len(RP_MAP)):
            _S[i][j] = hamming(_I[:, i] , _I[:, j])
    S[direction] = _S

# Stability

In [19]:
delta = {} # Stability matrix for assigning new cluster centroid

In [20]:
for direction in DIRECTIONS:
    _psi = np.sum(radio_map[direction], axis = 2) / TIMESTEPS
    # print(_psi.shape)
    _delta = np.sum((radio_map[direction] - _psi[:, :, np.newaxis]) ** 2 , axis = 2) / (TIMESTEPS-1)
    _delta = np.sum(np.nan_to_num(_delta) * I[direction] , axis = 0) * (1 / (np.sum(I[direction] , axis = 0)))
    # print(_delta)
    delta[direction] = _delta



/var/folders/48/z568rjb95513g5z3q1_tlmbm0000gn/T/ipykernel_77993/3170060250.py:5: RuntimeWarning: divide by zero encountered in true_divide
  _delta = np.sum(np.nan_to_num(_delta) * I[direction] , axis = 0) * (1 / (np.sum(I[direction] , axis = 0)))
/var/folders/48/z568rjb95513g5z3q1_tlmbm0000gn/T/ipykernel_77993/3170060250.py:5: RuntimeWarning: invalid value encountered in multiply
  _delta = np.sum(np.nan_to_num(_delta) * I[direction] , axis = 0) * (1 / (np.sum(I[direction] , axis = 0)))


# Clustering

In [21]:
ETA = 0.01 #Threshold for similarity for a node to belong to a cluster

In [22]:
CH = collections.defaultdict(lambda : collections.defaultdict(set))
FL = collections.defaultdict(lambda : collections.defaultdict(set))

for direction in ['north']:
    B = set([i for i in range(len(RP_MAP))])
    _S = S[direction]
    k = -1
    while B:
        k += 1
        fl = set()
        node = B.pop()
        for j in B:
            if j != node and _S[j,node] >= ETA:
                fl.add(j)
        B = B - fl
        CH[direction][k] = set([node])
        FL[direction][k] = fl

    K = k
    for k in range(K+1):
        cluster = list(CH[direction][k] | FL[direction][k])
        stabilities = delta[direction][cluster]
        CH[direction][k] = set([cluster[np.argmin(stabilities)]])
        FL[direction][k] = set(cluster) - CH[direction][k]

